# DIY Covid-19 Dashboard Introduction


With the normalization of the Covid-19, citizen especially London which has high population density should always pay some attention to the detection rate of the new cases. This dashboard is designed for showing the updated data of new cases number and new tests number in London. Users can choose the data you want to monitor to get individual chart and click the update button to get the latest data from PHE server. The dashboard can be presented by the scale of logarithm.


In [40]:
#import all the modules that is needed in the dashboard
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [41]:
# make figures larger
%matplotlib inline 
plt.rcParams['figure.dpi'] = 100

In [42]:
# Load initial data from local JSON files
with open("london.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [43]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

#This function wrangle the raw data into a DataFrame that will be used for plotting
def wrangle_data(rawdata):
    """ Wrangle the data into the format that is suitable for DataFrame"""
    datalist=jsondata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    #decode the time format into the format that DataFrame can read
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #to fill up the DataFrame
    index=pd.date_range(startdate, enddate, freq='D')
    londondf=pd.DataFrame(index=index, columns=['cases', 'tests'])
    for entry in datalist:
        date=parse_date(entry['date'])
        # to ganrantee the data is vaild
        for column in ['cases', 'tests']:
            if pd.isna(londondf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0 
                #replace None with 0 
                londondf.loc[date, column]=value
    #fill up any missing dates
    londondf.fillna(0.0, inplace=True)
    return(londondf)
df=wrangle_data(jsondata)

In [44]:
 def access_api():
    """ Accesses the PHE API  """
    #select the data we want
    filters = [
    'areaType=region',
    'areaCode=E12000007',
    'areaName=London'
    ]
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "tests": "newPCRTestsBySpecimenDate",    
    }
    #download the latest data by accessing api
    api = Cov19API(filters=filters, structure=structure)
    london=api.get_json()
    return (london)

In [45]:
def api_button_callback(button):
    """ Update and wrangle data with the feedback of clicking the button """
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    #give user feedback of clicking button
    apibutton.icon="check"
    apibutton.disabled=True
    

#design the update button
apibutton=wdg.Button(
    description='Update Data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cilck to updata latest data from NHS server",
    icon='download'
)

#connect the button with function
apibutton.on_click(api_button_callback)
display(apibutton)

Button(button_style='info', description='Update Data', icon='download', style=ButtonStyle(), tooltip='Cilck to…

In [46]:
#the two control panel to adjust the series and scale    
series=wdg.SelectMultiple(
    options=['cases', 'tests'],
    value=['cases', 'tests'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)   

def refresh_graph(gcols, gscale):
    """ This function change the value of the widget in order to force a redraw of the graph """
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        #give user instruction if they choose none of data
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(refresh_graph, {'gcols': series, 'gscale': scale})
#the layout of dashboard
controls=wdg.VBox([series, scale])
form=wdg.HBox([graph, controls])
display(form)